In [ ]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import pyarrow.feather as feather
import time
from datetime import datetime
from datetime import timedelta
import seaborn as sns


In [ ]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [ ]:
export_df["event_timestamp"]=pd.to_datetime(export_df["event_timestamp"]*1000, unit="ns")

In [ ]:
export_df["notif_viewed_ontime"]=list(map(lambda x,y,z: '' if x=="notification_received" else (1 if y < z else 0), export_df["event_name"], export_df["event_timestamp"],export_df["closed_at"])) #1 if the user view it on time  0 if not  and null if hasnt yet seen it 

In [ ]:
export_df["reaction_time"]=list(map(lambda x,y,z,w: y-w
if (x==1) else (y-z if (x==0) else 0), 
export_df["notif_viewed_ontime"],export_df["event_timestamp"],export_df["closed_at"],export_df["created_at"]))

In [ ]:
# IMPORTANT: REACTION TIME FOR THE CASES OF NOTIF VIEWED ONTIME = 0 IS THE DIFFERENCE BETWEEN INCIDENT CLOSED AND INCIDENT_INTERACTION
# NOTIF VIEWED ONTIME= 1 IS DIFF BETWEEN INCIDENT CREATED AND INCIDENT INTERACTION

PLOT OF REACTION TIME VS OPEN/DISMISSED FOR ACTIVE USERS
PLOT OF 

In [ ]:

active_df=export_df[export_df["notif_viewed_ontime"]==1].reset_index(drop=True)
unactive_df=export_df[export_df["notif_viewed_ontime"]==0].reset_index(drop=True)
#active_df.drop('notif_viewed_ontime', axis=1, inplace=True)
#unactive_df.drop('notif_viewed_ontime', axis=1, inplace=True)

In [ ]:
export_df.info()

# ASK TIM

In [41]:
#ask tim
unactive_analysis_df
#unactive_analysis_df['reaction_time'] = unactive_analysis_df['reaction_time'].apply(lambda x: x.value)
#unactive_analysis_df[["reaction_time","notif_viewed_ontime","event_name"]].head()
#sns.scatterplot(data=unactive_analysis_df[["reaction_time","notif_viewed_ontime","event_name"]], x= "reaction_time",y= "notif_viewed_ontime", hue="event_name")

,event_timestamp,event_name,user_id,created_at,published_at,closed_at,cause,effect,description,area_of_effect_coordinates_latitude,area_of_effect_coordinates_longitude,notif_viewed_ontime,reaction_time
0,2021-05-29 04:16:36.618000,notification_dismiss,8c7b7,2021-05-29 02:07:09,2021-05-29 02:07:12,2021-05-29 03:01:00,VehicularAccident,Delays,⌛ ¡No de nuevo! Hay retrasos por accidente ve...,-99.170952,19.393843,0,4536618000000
1,2021-05-29 04:16:36.622001,notification_dismiss,8c7b7,2021-05-29 03:17:01,2021-05-29 03:17:12,2021-05-29 04:01:00,Incident,Delays,⏳ ¡Pa variar! Usuarios reportan retrasos de ha...,NaN,NaN,0,936622001000
2,2021-05-29 17:14:09.994000,notification_dismiss,8c7b7,2021-05-29 13:12:27,2021-05-29 13:12:42,2021-05-29 14:01:00,Incident,Delays,⏳ ¡No te esponjes! Usuarios reportan retrasos ...,NaN,NaN,0,11589994000000
3,2021-05-29 03:17:06.023000,notification_dismiss,3f1e8,2021-05-29 01:21:13,2021-05-29 01:21:34,2021-05-29 02:17:00,BrokenVehicle,Delays,🚗 ¡Sopas! Retrasos por vehículo descompuesto e...,-99.066870,19.404783,0,3606023000000
4,2021-05-29 01:27:23.251000,notification_dismiss,81354,2021-05-28 20:37:32,2021-05-28 20:38:12,2021-05-28 21:03:00,VehicularAccident,Delays,⌛ ¡Ah jijo! Hay retrasos por accidente vehicul...,-99.082618,19.421229,0,15863251000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381734,2021-07-06 12:16:54.144002,notification_opened,b6c41,2021-07-06 10:58:23,2021-07-06 10:58:26,2021-07-06 12:00:00,Incident,Delays,⏳ ¡Changos! Usuarios informan retrasos de más ...,NaN,NaN,0,1014144002000
381735,2021-07-06 13:04:46.270002,notification_opened,b6c41,2021-07-06 12:19:18,2021-07-06 12:19:25,2021-07-06 12:50:00,VehicularAccident,LaneReduction,⏳ ¡Chanfle! Reducción de carriles por percance...,-99.059346,19.304670,0,886270002000
381736,2021-07-05 23:05:16.648000,notification_opened,7a79c,2021-07-05 21:39:17,2021-07-05 21:39:20,2021-07-05 22:40:01,Incident,Delays,"⏳ No te agüites, pero usuarios reportan retras...",NaN,NaN,0,1515648000000
381737,2021-07-06 17:08:21.329002,notification_opened,0bf97,2021-07-06 16:47:02,2021-07-06 16:47:39,NaT,Demonstration,Delays,✋ ¡Chanclas! Hay cierre de circulación por man...,-99.041880,19.343449,0,-9223372036854775808


# Analyzing agencies

In [ ]:
#Deeper understanding on agencies. Choosing aggfunc sum, as we want to know how many users are affected by their notifications. Observing values for both active and unactive users, it does not matter, as this is addressed in another part.
agencies_by_cause = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['cause'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')
agencies_by_effect = pd.pivot_table(export_df, values=['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg','agency_HE59N3RXM0q5vKu4AXlQZg','agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ','agency_MgUq5b9mOEunx6u4AXt_BA','agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw','agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
'agency_zCy9zG00HEqGeKu4AWZYNQ'], index=['effect'], aggfunc=np.sum, fill_value=0, margins=True, margins_name= 'Total')

In [ ]:
#Agencies MgUq5b9mOEunx6u4AXt_BA and V2AIQQKgmUO3VazvAOA-Cw created no incidents, dropping them
agencies_by_effect.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)
agencies_by_cause.drop(["agency_MgUq5b9mOEunx6u4AXt_BA","agency_V2AIQQKgmUO3VazvAOA-Cw"],axis=1, inplace=True)

# Unactive users

In [ ]:
unactive_df

In [ ]:
unactive_df.columns

In [ ]:
unactive_analysis_df=unactive_df.drop(['agency_GewRJAw5tUmC4Ku4AX1-SQ', 'agency_GtvOEQAFZ0GtU6u4AXwvPg',
       'agency_HE59N3RXM0q5vKu4AXlQZg', 'agency_JUR9bFXmVkWDHqu4AXaY0g',
       'agency_JfA8Bw8Zp024Kqu4AXiSpQ', 'agency_MgUq5b9mOEunx6u4AXt_BA',
       'agency_NuuRQ2I1Q0a50Kv-AVKlLA', 'agency_V2AIQQKgmUO3VazvAOA-Cw',
       'agency_jLjibFoim0iwWau4AWoEdQ', 'agency_pky7jovXYkaw-awAAMrQ3g',
       'agency_zCy9zG00HEqGeKu4AWZYNQ', 'surrogate_id','document_id'], axis=1)

In [ ]:
unactive_analysis_df

In [ ]:
# Filtering new users

In [ ]:
export_df.tail()